## Cupac tutorial

In this example we demonstrate usage of CUPAC as a way to increase power.

We compare vainilla (non-cupac) gee estimation with gee estimation with cupac.

In [3]:
from datetime import date

import numpy as np
import pandas as pd
from cluster_experiments import GeeExperimentAnalysis
from cluster_experiments import UniformPerturbator
from cluster_experiments import PowerAnalysis
from cluster_experiments import ClusteredSplitter
from sklearn.ensemble import HistGradientBoostingRegressor


def generate_random_data(clusters, dates, N):
    """Generates target as a non-linear function of the covariates, the cluster mean and some residual"""

    # Every cluster has a mean
    df_clusters = pd.DataFrame(
        {
            "cluster": clusters,
            "cluster_mean": np.random.normal(0, 0.1, size=len(clusters)),
        }
    )
    # The target is the sum of: covariates, cluster mean and random residual
    df = (
        pd.DataFrame(
            {
                "cluster": np.random.choice(clusters, size=N),
                "residual": np.random.normal(0, 1, size=N),
                "date": np.random.choice(dates, size=N),
                "x1": np.random.normal(0, 1, size=N),
                "x2": np.random.normal(0, 1, size=N),
                "x3": np.random.normal(0, 1, size=N),
                "x4": np.random.normal(0, 1, size=N),
            }
        )
        .merge(df_clusters, on="cluster")
        .assign(
            target=lambda x: x["x1"] * x["x2"]
            + x["x3"] ** 2
            + x["x4"]
            + x["cluster_mean"]
            + x["residual"]
        )
    )

    return df

### Data generation

We have data that contains:
* Cluster
* Date
* Some covariates
* Outcome (target)

We want to run a switchback-clustered experiment for around 15 days.

We take data from day 15 to day 31 as the analysis data.

We take data from day 1 to day 14 as the pre-analysis data, which is going to be used to fit the cupac model.

In [4]:
clusters = [f"Cluster {i}" for i in range(100)]
dates = [f"{date(2022, 1, i):%Y-%m-%d}" for i in range(1, 32)]
experiment_dates = [f"{date(2022, 1, i):%Y-%m-%d}" for i in range(15, 32)]
N = 10_000
df = generate_random_data(clusters, dates, N).drop(columns=["residual", "cluster_mean"])
df_analysis = df.query(f"date.isin({experiment_dates})")
df_pre = df.query(f"~date.isin({experiment_dates})")
df

,cluster,date,x1,x2,x3,x4,target
0,Cluster 30,2022-01-28,-1.356347,0.111188,0.656200,0.112841,0.412073
1,Cluster 30,2022-01-03,0.269740,0.840074,-0.393097,1.170404,0.760463
2,Cluster 30,2022-01-15,1.216208,-0.241652,0.471189,1.018055,1.184657
3,Cluster 30,2022-01-30,1.827100,-1.568052,-0.647234,-0.850376,-3.334617
4,Cluster 30,2022-01-16,-0.113764,-0.631820,-1.286758,2.140398,3.208504
...,...,...,...,...,...,...,...
9995,Cluster 59,2022-01-01,-0.458435,1.316754,-0.761421,-1.070210,-1.869195
9996,Cluster 59,2022-01-10,1.080839,1.162758,0.003099,0.197876,0.987346
9997,Cluster 59,2022-01-27,-0.028537,-0.653719,-0.237255,1.852296,0.975927
9998,Cluster 59,2022-01-29,-0.251758,-0.993584,-1.620943,-0.303710,2.714441


### Vainilla method

Just run a regular gee estimation, no covariate adjustment.

In [5]:
# Splitter and perturbator
sw = ClusteredSplitter(
    cluster_cols=["cluster", "date"],
)

perturbator = UniformPerturbator(
    average_effect=0.1,
)

# Vainilla GEE
analysis = GeeExperimentAnalysis(
    cluster_cols=["cluster", "date"],
)
pw_vainilla = PowerAnalysis(
    perturbator=perturbator,
    splitter=sw,
    analysis=analysis,
    n_simulations=50,
)

power = pw_vainilla.power_analysis(df_analysis)
print(f"Not using cupac: {power = }")


Not using cupac: power = 0.5


We can see that power is not great

### Cupac method

Use GBM model with covariates x1, x2, x3, x4 fitted on pre-analysis data; use the predictions of this model on analysis data to reduce variance.

In [6]:
# Cupac GEE
analysis = GeeExperimentAnalysis(
    cluster_cols=["cluster", "date"], covariates=["estimate_target"]
)

gbm = HistGradientBoostingRegressor()
pw_cupac = PowerAnalysis(
    perturbator=perturbator,
    splitter=sw,
    analysis=analysis,
    n_simulations=50,
    cupac_model=gbm,
    features_cupac_model=["x1", "x2", "x3", "x4"],
)

power = pw_cupac.power_analysis(df_analysis, df_pre)
print(f"Using cupac: {power = }")

Using cupac: power = 0.9


Power has increased!